In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns


data = pd.read_excel('courses.xlsx')
questions = {
    'sql for data analysis': {
        'beginner': [
            ("1.how to create new table in DB?", ["CREATE", "UPDATE", "NEW", "DELETE"], "CREATE"),
            ("2.how to remove table in DB?", ["REMOVE", "DELETE", "DROP", "NEW"], "DROP"),
            ("3.which statement allows to read data and display it?", ["SELECT", "READ", "DISPLAY", "UPDATE"], "SELECT"),
            ("4.how to write select statement?", ["SELECT C_NAME FROM T_NAME", "SELECT *FROM T_NAME", "SELECT FROM", "SELECT_ALL"], "SELECT C_NAME FROM T_NAME"),
            ("5.how to display the first 10 rows?", ["head 10", "limit 10", "top 10", "head -10"], "limit 10"),
            ("6.how to sort values in sql?",["NEW","SORT BY","ORDER BY","CREATE"],"ORDER BY")
            
        ],
        'intermediate': [
            ("7......allows you to filter a set of results based on specific criteria?",["AND","FILTER","WHERE","EQUAL"],"WHERE"),
            ("8. WHERE clause goes after FROM but before ORDER BY or LIMIT?", ["FALSE", "TRUE"], "TRUE"),
            ("9. WHERE clause can be used with non-numeric data?", ["TRUE", "FALSE"], "TRUE"),
            ("10. A new column that is manipulated from the existing column in your DB?", ["DERIVED", "MANIPULATED", "EXISTED"], "DERIVED"),
            ("11. We use .... to filter similar to a specific value", ["FILTER", "SEARCH", "SPECIFIC", "LIKE"], "LIKE"),
            ("12. What allows you to filter data based on several specific values?", ["IN", "WHERE", "FILTER", "LIKE"], "IN")
        ]
    },
    'python': {
        'beginner': [
            ("1. How do you print something in Python?", ["echo", "display", "print", "output"], "print"),
            ("2. What is used to create a list in Python?", ["()", "{}", "[]", "||"], "[]"),
            ("3. Which keyword is used to define a function?", ["def", "function", "create", "fn"], "def")
        ],
        'intermediate': [
             ("1. How to display the first 10 rows of a DataFrame?", ["head 10", "limit 10", "top 10", "head(-10)"], "head 10"),
            ("2. How to sort values in a Pandas DataFrame?", ["sort_values", "order_by", "sort", "arrange"], "sort_values")
        ]
    }
}

# Function to ask a question
def ask_question(question, options, correct_option):
    print(question)
    for i, option in enumerate(options, 1):
        print(f"{i}. {option}")
    
    while True:
        try:
            answer = int(input("Choose the correct option (1-4): "))
            if 1 <= answer <= len(options):
                break
            else:
                print("Invalid choice. Please choose a number between 1 and 4.")
        except ValueError:
            print("Invalid input. Please enter a number.")
    
    return options[answer - 1] == correct_option


track = input("Choose a track (sql for data analysis/python): ").lower()
level = input("Choose a level (beginner/intermediate/advanced): ").lower()

#get track,level
selected_questions = questions.get(track, {}).get(level, [])
if not selected_questions:
    print("No questions found for the selected track and level.")
    exit()

# Filter data based on track and level
filtered_data = data[(data['track'] == track) & (data['level'] == level)]


# Ask the questions and update the DataFrame
for index, (question, options, correct_option) in enumerate(selected_questions):
    is_correct = ask_question(question, options, correct_option)
    

data_index = filtered_data.index[index]
if is_correct:
    data.loc[data_index, 'completed'] = 1
    print("Correct Answer!")
else:
    data.loc[data_index, 'completed'] = 0
    print("Incorrect. The correct answer is:", correct_option)
        
#visualization
overall_completion = data['completed'].value_counts()
labels = ['Completed' if val == 1 else 'Not Completed' for val in overall_completion.index]
plt.figure(figsize=(8, 8))
plt.pie(overall_completion, labels=labels, autopct='%1.1f%%', startangle=90, colors=['#FFC0CB', '#808080'])
plt.title('Overall Completion Status')
plt.show()


filtered_data = data[(data['track'] == track) & (data['level'] == level)]
#print(filtered_data[['dscription','url', 'completed']])

# Save
data.to_excel('courses.xlsx', index=False)
print("Results saved successfully.")

# One-hot encode the URL column to prepare for clustering
encoder = OneHotEncoder()
encoded_urls = encoder.fit_transform(data[['url']])

# Add clustering results to the DataFrame
def cluster_data(track, level, data, max_clusters=3):
    # Filter data for the specific track and level
    filtered_data = data[(data['track'] == track) & (data['level'] == level)]
    if filtered_data.empty:
        print(f"No data found for track '{track}' and level '{level}'.")
        return pd.DataFrame()
    
    n_samples = len(filtered_data)
    n_clusters = min(max_clusters, n_samples)  # Ensure n_clusters is <= n_samples
    
    if n_clusters < 2:
        print("Not enough data to form clusters.")
        return pd.DataFrame()
    
    # Cluster URLs based on their completion status
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    filtered_data = filtered_data.copy()
    filtered_data['cluster'] = kmeans.fit_predict(encoded_urls[filtered_data.index])
    
    return filtered_data

#recommend URLs
def recommend_urls(track, level, data, max_clusters=3):
    # Cluster data for the given track and level
    clustered_data = cluster_data(track, level, data, max_clusters)
    if clustered_data.empty or 'cluster' not in clustered_data.columns:
        return data[(data['track'] == track) & 
                    (data['level'] == level) & 
                    (data['completed'] == 0)]['url'].unique()
    
    # Recommend URLs where 'completed' is 
    recommendations = clustered_data[clustered_data['completed'] == 0]['url'].unique()
    return recommendations

# Get recommendations
recommendations = recommend_urls(track, level, data)
if recommendations.size > 0:
    print("Recommended URLs for you:")
    print(recommendations)
else:
    print("congratulations you have passed this level.")

In [ ]:
#import libraries
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#import dataset
data = pd.read_excel('courses.xlsx')


In [ ]:
#set questions
questions = {
    'sql for data analysis': {
        'beginner': [
            ("1.how to create new table in DB?", ["CREATE", "UPDATE", "NEW", "DELETE"], "CREATE"),
            ("2.how to remove table in DB?", ["REMOVE", "DELETE", "DROP", "NEW"], "DROP"),
            ("3.which statement allows to read data and display it?", ["SELECT", "READ", "DISPLAY", "UPDATE"], "SELECT"),
            ("4.how to write select statement?", ["SELECT C_NAME FROM T_NAME", "SELECT *FROM T_NAME", "SELECT FROM", "SELECT_ALL"], "SELECT C_NAME FROM T_NAME"),
            ("5.how to display the first 10 rows?", ["head 10", "limit 10", "top 10", "head -10"], "limit 10"),
            ("6.how to sort values in sql?",["NEW","SORT BY","ORDER BY","CREATE"],"ORDER BY")
            
        ],
        'intermediate': [
            ("7......allows you to filter a set of results based on specific criteria?",["AND","FILTER","WHERE","EQUAL"],"WHERE"),
            ("8. WHERE clause goes after FROM but before ORDER BY or LIMIT?", ["FALSE", "TRUE"], "TRUE"),
            ("9. WHERE clause can be used with non-numeric data?", ["TRUE", "FALSE"], "TRUE"),
            ("10. A new column that is manipulated from the existing column in your DB?", ["DERIVED", "MANIPULATED", "EXISTED"], "DERIVED"),
            ("11. We use .... to filter similar to a specific value", ["FILTER", "SEARCH", "SPECIFIC", "LIKE"], "LIKE"),
            ("12. What allows you to filter data based on several specific values?", ["IN", "WHERE", "FILTER", "LIKE"], "IN")
        ]
    },
    'python': {
        'beginner': [
            ("1. How do you print something in Python?", ["echo", "display", "print", "output"], "print"),
            ("2. What is used to create a list in Python?", ["()", "{}", "[]", "||"], "[]"),
            ("3. Which keyword is used to define a function?", ["def", "function", "create", "fn"], "def")
        ],
        'intermediate': [
             ("1. How to display the first 10 rows of a DataFrame?", ["head 10", "limit 10", "top 10", "head(-10)"], "head 10"),
            ("2. How to sort values in a Pandas DataFrame?", ["sort_values", "order_by", "sort", "arrange"], "sort_values")
        ]
    }
}

In [ ]:
#ask a question
def ask_question(question, options, correct_option):
    print(question)
    for i, option in enumerate(options, 1):
        print(f"{i}. {option}")
    
    while True:
        try:
            answer = int(input("Choose the correct option (1-4): "))
            if 1 <= answer <= len(options):
                break
            else:
                print("Invalid choice. Please choose a number between 1 and 4.")
        except ValueError:
            print("Invalid input. Please enter a number.")
    
    return options[answer - 1] == correct_option

In [ ]:
#let user select track &level
track = input("Choose a track (sql for data analysis/python): ").lower()
level = input("Choose a level (beginner/intermediate/advanced): ").lower()

#get track,level
selected_questions = questions.get(track, {}).get(level, [])
if not selected_questions:
    print("No questions found for the selected track and level.")
    exit()

# Filter data based on track and level
filtered_data = data[(data['track'] == track) & (data['level'] == level)]


# Ask the questions and update the DataFrame
for index, (question, options, correct_option) in enumerate(selected_questions):
    is_correct = ask_question(question, options, correct_option)

In [ ]:
#set 1,0 in completed column according to user answer
data_index = filtered_data.index[index]
if is_correct:
    data.loc[data_index, 'completed'] = 1
    print("Correct Answer!")
else:
    data.loc[data_index, 'completed'] = 0
    print("Incorrect. The correct answer is:", correct_option)

In [ ]:
#visualization
overall_completion = data['completed'].value_counts()
labels = ['Completed' if val == 1 else 'Not Completed' for val in overall_completion.index]
plt.figure(figsize=(8, 8))
plt.pie(overall_completion, labels=labels, autopct='%1.1f%%', startangle=90, colors=['#FFC0CB', '#808080'])
plt.title('Overall Completion Status')
plt.show()

In [ ]:
# One-hot encode the URL column to prepare for clustering
encoder = OneHotEncoder()
encoded_urls = encoder.fit_transform(data[['url']])

# Add clustering results to the DataFrame
def cluster_data(track, level, data, max_clusters=3):
    # Filter data for the specific track and level
    filtered_data = data[(data['track'] == track) & (data['level'] == level)]
    if filtered_data.empty:
        print(f"No data found for track '{track}' and level '{level}'.")
        return pd.DataFrame()
    
    # Adjust the number of clusters based on the size of the filtered data
    n_samples = len(filtered_data)
    n_clusters = min(max_clusters, n_samples)  # Ensure n_clusters is <= n_samples
    
    if n_clusters < 2:
        print("Not enough data to form clusters.")
        return pd.DataFrame()
    
    # Cluster URLs based on their completion status
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    filtered_data = filtered_data.copy()
    filtered_data['cluster'] = kmeans.fit_predict(encoded_urls[filtered_data.index])
    
    return filtered_data

In [ ]:
#recommend URLs
def recommend_urls(track, level, data, max_clusters=3):
    # Cluster data for the given track and level
    clustered_data = cluster_data(track, level, data, max_clusters)
    if clustered_data.empty or 'cluster' not in clustered_data.columns:
        return data[(data['track'] == track) & 
                    (data['level'] == level) & 
                    (data['completed'] == 0)]['url'].unique()
    
    
    recommendations = clustered_data[clustered_data['completed'] == 0]['url'].unique()
    return recommendations

In [ ]:
#display recommended remaining courses
# Get recommendations
recommendations = recommend_urls(track, level, data)
if recommendations.size > 0:
    print("Recommended URLs for you:")
    print(recommendations)
else:
    print("No recommendations available.")